In [1]:
import os

import pandas as pd
from tqdm import tqdm

In [2]:
keywords_gt = pd.read_csv(os.path.join('..', 'data', 'keywords_gt_with_examples.csv'))

In [3]:
data = keywords_gt[
    (keywords_gt['n_keywords'] == 2) & 
    (keywords_gt['presence'] == 1) &
    (keywords_gt['result_count'] >= 15) & 
    (keywords_gt['r_id_index'] < 20)
].iloc[:1].reset_index(drop=True)

data

,country_tag,type,difficulty,r_rsc_id,s_rsc_id,r_pkg_id,s_pkg_id,r_col_name,s_col_name,nl,top_k,n_keywords,keywords,presence,result_count,r_id_index,s_id_index,prompt_tokens,completion_tokens,response
0,CAN,multi-table-join,challenging,a7c95445-e336-41e3-945c-1bc938ff0612,ae2204c7-db9e-450e-8158-c70eedada7ae,12c89aa5-063a-f0ed-334a-3019133b9ee8,4879e498-c9ac-44a3-d7d1-21af22e2224c,department_/_minista_re,department_/_minista_re,How has the payment amount changed for the sam...,1000,2,suppliers+payments,1,17,14.0,12.0,648,15,<think>\n\n</think>\n\n<keywords>suppliers+pay...


## Fetching Packages and relative Tables

In [8]:
from ckan import CanadaCKAN

ckan_client = CanadaCKAN() 

In [9]:
import os
import shutil

shutil.rmtree('tmp')
os.makedirs('tmp', exist_ok=True)

cnt = ckan_client.download_tables_from_package_search('tmp', 'csv', 3, q='suppliers+payments', rows=20, defType='edismax', sort='sort desc')

cnt

https://open.canada.ca/data/api/3/action/package_search?q=suppliers+payments&rows=20&defType=edismax&sort=sort desc


53

In [10]:
resource_ids = list(map(lambda s: s.removesuffix('.csv'), os.listdir('tmp')))

data['r_rsc_id'][0] in resource_ids, data['s_rsc_id'][0] in resource_ids

(True, True)

## Setup the BLEND index with ALL the downloaded resources

In [11]:
import blend

indexer = blend.BLEND('blend.db')

In [12]:
_, dbcon = indexer.create_index('tmp', max_workers=6, limit_rows_per_table=1000, verbose=True)

┌───────────┬─────────┬──────────┬────────┬──────────────────┬─────────────┬──────────┬──────────┐
│ CellValue │ TableId │ ColumnId │ RowId  │ GranularityClass │ Granularity │ Quadrant │ SuperKey │
│  varchar  │ varchar │  uint32  │ uint32 │     varchar      │   varchar   │ boolean  │   blob   │
├───────────┴─────────┴──────────┴────────┴──────────────────┴─────────────┴──────────┴──────────┤
│                                             0 rows                                             │
└────────────────────────────────────────────────────────────────────────────────────────────────┘

None


Parsing and storing tables: 100%|██████████| 53/53 [00:01<00:00, 46.05it/s]


Tables ingestion completed.
Creating indexes...
Index creation completed.


In [13]:
dbcon.table('AllTables').show()

AttributeError: 'NoneType' object has no attribute 'table'

In [ ]:
dbcon.sql("SELECT COUNT(*) FROM AllTables;")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│         2953 │
└──────────────┘

In [ ]:
s_rsc_metadata = ckan_client.resource_show(resource_id=data['s_rsc_id'][0])

In [ ]:
response = ckan_client.package_search("payment+supplier", 3)

## Setup an Agent for Joinable Column Selection

In [4]:
import os

from dotenv import load_dotenv
from autogen_core.models import UserMessage
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import StructuredMessage
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

In [5]:
# same seed for any random operation
seed = 42

# Load environment variables from .env file
load_dotenv()
assert "TOGETHER_API_KEY" in os.environ

settings = {
    "model": "Qwen/Qwen3-235B-A22B-fp8-tput",
    "temperature": 0.1,
    "top_p": 1,
    "frequency_penalty": 0,
    "presence_penalty": 0,
    "max_tokens": 10000,
}

model_client = OpenAIChatCompletionClient(
    base_url = "https://api.together.xyz/v1/",
    api_key=os.getenv("TOGETHER_API_KEY", ""),
    model_info= {
        "vision": False,
        "function_calling": True,
        "json_output": True,
        "family": "unknown",
        "structured_output": True
    },
    **settings   
)

In [22]:
with open(os.path.join('..', 'backend', 'prompts', 'TablePreFilter.md')) as file:
    TABLE_PRE_FILTER_PROMPT_TEMPLATE = file.read()

In [7]:
data

,country_tag,type,difficulty,r_rsc_id,s_rsc_id,r_pkg_id,s_pkg_id,r_col_name,s_col_name,nl,top_k,n_keywords,keywords,presence,result_count,r_id_index,s_id_index,prompt_tokens,completion_tokens,response
0,CAN,multi-table-join,challenging,a7c95445-e336-41e3-945c-1bc938ff0612,ae2204c7-db9e-450e-8158-c70eedada7ae,12c89aa5-063a-f0ed-334a-3019133b9ee8,4879e498-c9ac-44a3-d7d1-21af22e2224c,department_/_minista_re,department_/_minista_re,How has the payment amount changed for the sam...,1000,2,suppliers+payments,1,17,14.0,12.0,648,15,<think>\n\n</think>\n\n<keywords>suppliers+pay...


In [9]:
from ckan import CanadaCKAN

ckan_client = CanadaCKAN() 

ckan_response = ckan_client.package_search(q='suppliers+payments', rows=20, defType='edismax', sort='sort desc')

https://open.canada.ca/data/api/3/action/package_search?q=suppliers+payments&rows=20&defType=edismax&sort=sort desc


In [10]:
data['nl'][0]

'How has the payment amount changed for the same suppliers between fiscal year 2020 and 2023 in different departments? Could you show me the department name, supplier name, payments from both years, and the difference in payment amounts? Please focus on suppliers that appear in both years.'

In [18]:
model_choices = []

In [23]:
no_think = True

for package in tqdm(ckan_response['result']['results']):
    # print(package['title'])
    # print(package['notes'])
    # print(', '.join(package['keywords']['en']))
    # print()
    prompt = TABLE_PRE_FILTER_PROMPT_TEMPLATE.format(
        question=data['nl'][0],
        title=package['title'],
        notes='N/A', # package['notes'][:300],
        keywords='N/A' # ', '.join(package['keywords']['en'])
    )

    if no_think:
        prompt = '/nothink' + prompt

    analyzer = AssistantAgent(
    name="TablePreFilter", model_client=model_client, system_message=prompt
)

    response = await analyzer.run()
    response = response.messages[-1]
    model_choices.append(response)


100%|██████████| 17/17 [00:16<00:00,  1.05it/s]


In [24]:
data['s_pkg_id']

0    4879e498-c9ac-44a3-d7d1-21af22e2224c
Name: s_pkg_id, dtype: object

In [41]:
import re

answers = [
    re.search(r"<answer>(YES|NO)</answer>", message.content).group(1)
    for message in model_choices
]

answers = list(zip(answers, [p['id'] for p in ckan_response['result']['results']]))
answers = [a[1] for a in answers if a[0] == 'YES']
answers

['f6bad1d5-d1dc-4fdd-a076-650f55cc3fbc',
 '3d48d117-64a0-e504-085d-159c69ee51da',
 '804a1d01-7ec3-a54c-022f-faf471350661']

In [43]:
for package in tqdm(ckan_response['result']['results']):
    if package['id'] == 'f6bad1d5-d1dc-4fdd-a076-650f55cc3fbc' or True:
        print(package['id'])
        print(package['title'])
        # print(package['notes'][:300])
    if package['id'] == '12c89aa5-063a-f0ed-334a-3019133b9ee8':
        print()
    # print(', '.join(package['keywords']['en']))
    # print()


100%|██████████| 17/17 [00:00<00:00, 29796.56it/s]

a5367f15-3f30-4d2b-91df-40275513788f
Difficulty of obstacles when exporting or attempting to export goods or services, by industry and enterprise size
eaaef3a6-1100-fa3b-5c05-f4b97b0fab1e
2021 - Combined Supplier and Grant Payments
f6bad1d5-d1dc-4fdd-a076-650f55cc3fbc
CRF Detailed Schedules of Payments - Other Supplier Payments
19cc9559-b2da-e0a5-f269-362cee1a3612
Grant payments over $250,000 2014-2015
a9103f31-f16b-a7a2-d056-68a685155c22
2021 - Supplier and Grant Payments by Department
8be07a6b-2f19-48e5-a4ba-024e5e4933c5
Confidence level of business or organization in its ability to make payments to suppliers and service providers in full and on time, first quarter of 2025
b8189a86-fa88-4c1d-86e4-cc61ff549b71
Confidence level of business or organization in its ability to make payments to suppliers and service providers in full and on time, second quarter of 2025
2142e30d-9275-424d-be43-f5cc7ec91916
Aircraft Fleet: Monthly Fuel Cost and Consumption, by Region or Group
3d48d117-64a0-e5

In [71]:
data['nl'][0]

'How has the payment amount changed for the same suppliers between fiscal year 2020 and 2023 in different departments? Could you show me the department name, supplier name, payments from both years, and the difference in payment amounts? Please focus on suppliers that appear in both years.'

In [21]:
print(model_choices[-2].content)

<think>
Okay, let's see. The user is asking about changes in payment amounts for the same suppliers between fiscal years 2020 and 2023 across different departments. They want the department name, supplier name, payments from both years, and the difference. The key here is that the suppliers must appear in both years.

The available package is titled "2020 - Combined Supplier and Grant Payments". The notes and keywords are not available. So, the package is specific to 2020 and includes supplier payments. But the user is asking for data from both 2020 and 2023. Since the package only covers 2020, it doesn't include 2023 data. The question requires comparing two different years, so the package alone can't answer that. However, maybe if there's another package for 2023, but the current task only mentions this one package. Since the package is only for 2020, it can't provide the 2023 data needed for comparison. Therefore, the answer would be no because the package doesn't contain data for b